In [1]:
import os


In [2]:
%pwd

'/home/nanshibukawa/Documents/mestrado/disease-classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/nanshibukawa/Documents/mestrado/disease-classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [8]:
import os
import urllib.request as request
import zipfile
import gdown
from cnnClassifier.logger import configure_logger
from cnnClassifier.utils.common import get_size

logger = configure_logger(logger_name =__name__)



In [9]:
logger.error("Downloading file from the internet...")

2025-06-23 20:51:14 | ERROR    | __main__:<module>:1 | Downloading file from the internet...


In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            logger.exception(e)
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function return None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True
                    )
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)


2025-06-23 20:51:14 | INFO     | cnnClassifier.utils.common:read_yaml:35 | YAML file read successfully from config/config.yaml
2025-06-23 20:51:14 | INFO     | cnnClassifier.utils.common:read_yaml:47 | YAML file content: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1oT2VFbMnAScIhUcPpP1q0zjcWZtlSSv4/view?usp=sharing', 'source_URL_id': '1oT2VFbMnAScIhUcPpP1q0zjcWZtlSSv4', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
2025-06-23 20:51:14 | INFO     | cnnClassifier.utils.common:read_yaml:35 | YAML file read successfully from params.yaml
2025-06-23 20:51:14 | INFO     | cnnClassifier.utils.common:read_yaml:47 | YAML file content: {'key': 'value'}
2025-06-23 20:51:14 | INFO     | cnnClassifier.utils.common:create_directories:68 | Created directory: artifacts
2025-06-23 20:51:14 | INFO     | cnnClassifier.utils.common:create_directories:68 | Created 

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1oT2VFbMnAScIhUcPpP1q0zjcWZtlSSv4
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1oT2VFbMnAScIhUcPpP1q0zjcWZtlSSv4&confirm=t&uuid=0ba8aeb3-d66b-4b90-a8c4-29560a0b1988
To: /home/nanshibukawa/Documents/mestrado/disease-classification/artifacts/data_ingestion/data.zip
100%|██████████| 1.63G/1.63G [03:27<00:00, 7.86MB/s]

2025-06-23 20:54:44 | INFO     | __main__:download_file:19 | Downloaded data from https://drive.google.com/file/d/1oT2VFbMnAScIhUcPpP1q0zjcWZtlSSv4/view?usp=sharing into file artifacts/data_ingestion/data.zip
